# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port alfred,-33.5906,26.8910,71.19,92,97,5.99,ZA,1732549160
1,1,argo,37.6333,22.7333,51.03,39,0,1.66,GR,1732549161
2,2,hithadhoo,-0.6000,73.0833,82.83,76,100,14.12,MV,1732549162
3,3,kyaikkami,16.0800,97.5675,82.80,69,42,5.68,MM,1732549163
4,4,grytviken,-54.2811,-36.5092,37.42,75,100,1.88,GS,1732549165


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
#%%capture --no-display

# Configure the map plot
humidityPlot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.7,
    tiles = "OSM",
    frame_height = 504,
    frame_weight = 896,
    hover_cols = ["City", "Humidity"]

    
)

# Display the map
humidityPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
idealCityDF = city_data_df.loc[
    (city_data_df["Max Temp"] > 69.8) & (city_data_df["Max Temp"] < 80.6) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"]== 0)
]

# Drop any rows with null values
idealCityDF = idealCityDF.dropna()

# Display sample data
print(len(idealCityDF))
idealCityDF.head()

4


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
250,250,tamanrasset,22.7850,5.5228,76.91,14,0,3.44,DZ,1732549508
316,316,treasure island,27.7692,-82.7690,71.19,67,0,1.99,US,1732549614
368,368,mahina,-17.5065,-149.4890,76.77,83,0,2.30,PF,1732549696
556,556,mukalla,14.5425,49.1242,77.63,68,0,3.76,YE,1732549937


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealCityDF[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel name
250,tamanrasset,DZ,22.7850,5.5228,14,
316,treasure island,US,27.7692,-82.7690,67,
368,mahina,PF,-17.5065,-149.4890,83,
556,mukalla,YE,14.5425,49.1242,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 100000
params = {
    "categories": "accomodation.hotel", 
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row ["Lat"]
    longitude = row ["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude}, {latitude},{radius}"
    params["bias"] = f"proximity:{longitude}, {latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found"]
hotel_df

Starting hotel search
tamanrasset - nearest hotel: No hotel found
treasure island - nearest hotel: No hotel found
mahina - nearest hotel: No hotel found
mukalla - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel name,Hotel Name


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.